In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler

In [2]:
cancer_data = pd.read_csv("cancer.csv")
print(cancer_data.shape)
cancer_labels = cancer_data['label'].values
cancer_features = cancer_data.drop(['label'], axis=1).values
cancer_data.head()

(569, 31)


,label,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
spam_data = pd.read_csv("spam.csv")
print(spam_data.shape)
spam_labels = spam_data['label'].values
spam_features = spam_data.drop(['label'], axis=1).values
spam_data.head()

(4601, 58)


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_:,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,label
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [4]:
def dist(x1, x2):
    return np.linalg.norm(x1 - x2)


class KNN:
    def __init__(self, k=3):
        self.k = k
        
    def fit(self, X, y):
        self.data = X
        self.ys = y
        
    def predict(self, xs):
        return np.array([self.get_label(self.filter_neighbours(self.get_neighbours(x), self.k)) for x in xs])
    
    def get_neighbours(self, x):
        '''
        return list of pairs (ind, dist to x) sorted by dist
        '''
        dists = enumerate(map(lambda y: dist(x, y), self.data))
        return self.filter_neighbours(sorted(dists, key=lambda y: y[1]), self.k)
    
    def filter_neighbours(self, neighbours, k):
        raise NotImplementedError()
    
    def get_label(self, x):
        raise NotImplementedError()
        

class KNNClassifier(KNN):
    def __init__(self, k=3):
        super().__init__(k)
        
    def filter_neighbours(self, neighbours, k):
        return neighbours[:k]
        
    def get_label(self, neighbours):
        labels = {}
        for i, x in neighbours:
            if self.ys[i] not in labels:
                labels[self.ys[i]] = 0
            labels[self.ys[i]] += 1
        return sorted(list(labels.items()), key=lambda t: -t[1])[0][0]
    

class RNNClassifier(KNN):
    def __init__(self, k=3):
        super().__init__(k)
        
    def filter_neighbours(self, neighbours, r):
        return list(filter(lambda x: x[1] < r, neighbours))
        
    def get_label(self, neighbours):
        labels = {}
        for i, x in neighbours:
            if self.ys[i] not in labels:
                labels[self.ys[i]] = 0
            labels[self.ys[i]] += 1
        if len(labels) == 0:
            return self.ys[0]
        return sorted(list(labels.items()), key=lambda t: -t[1])[0][0]

In [5]:
def accuracy(y_pr, y_te):
    return np.mean(y_pr == y_te)

def evaluate_knn(X, y, ks, knn_constructor):
    loo = LeaveOneOut()
    match = np.zeros(len(ks))
    max_k = max(ks)
    for tr_ind, te_ind in loo.split(X):
        X_tr, X_te = X[tr_ind], X[te_ind]
        y_tr, y_te = y[tr_ind], y[te_ind]
        knn = knn_constructor(max_k)
        knn.fit(X_tr, y_tr)
        neighbours = knn.get_neighbours(X_te[0])
        for i, k in enumerate(ks):
            match[i] += 1 if knn.get_label(knn.filter_neighbours(neighbours, k)) == y_te[0] else 0
    return match / loo.get_n_splits(X)

In [6]:
print("kNN on cancer data with k in [1, 10]")
print(evaluate_knn(cancer_features, cancer_labels, list(range(1, 11)), KNNClassifier))

kNN on cancer data with k in [1, 10]
[0.91564148 0.91564148 0.92618629 0.92618629 0.93321617 0.92970123
 0.9314587  0.9314587  0.93321617 0.9314587 ]


In [7]:
print("kNN on spam data with k in [1, 10]")
print(evaluate_knn(spam_features, spam_labels, list(range(1, 11)), KNNClassifier))

kNN on spam data with k in [1, 10]
[0.83047164 0.83047164 0.81482286 0.82655944 0.81417083 0.82329928
 0.80417301 0.81764834 0.797218   0.81286677]


In [12]:
# Find radius on big interval like binary search
print("Radius Neighbours on cancer data with R in [1, 1000]")
print(evaluate_knn(cancer_features, cancer_labels, np.linspace(1, 1000, 10), RNNClassifier))

Radius Neighbours on cancer data with R in [1, 1000]
[0.37258348 0.91915641 0.91388401 0.89455185 0.88400703 0.86643234
 0.85764499 0.84358524 0.82073814 0.79964851]


In [13]:
print("Radius Neighbours on cancer data with R in [30, 250]")
print(evaluate_knn(cancer_features, cancer_labels, np.linspace(30, 250, 10), RNNClassifier))

Radius Neighbours on cancer data with R in [30, 35]
[0.91036907 0.92970123 0.92267135 0.92091388 0.91915641 0.91739895
 0.91564148 0.91212654 0.91564148 0.91036907]


In [14]:
print("Radius Neighbours on cancer data with R in [40, 80]")
print(evaluate_knn(cancer_features, cancer_labels, np.linspace(40, 80, 10), RNNClassifier))

Radius Neighbours on cancer data with R in [40, 80]
[0.92618629 0.92442882 0.9314587  0.92970123 0.9314587  0.92970123
 0.92970123 0.92618629 0.92267135 0.92442882]


In [15]:
print("Radius Neighbours on cancer data with R in [50, 60]. Best values achieved in this interval")
print(evaluate_knn(cancer_features, cancer_labels, np.linspace(50, 60, 10), RNNClassifier))

Radius Neighbours on cancer data with R in [50, 60]. Best values achieved in this interval
[0.93321617 0.93497364 0.9314587  0.92970123 0.92970123 0.92794376
 0.92970123 0.9314587  0.92794376 0.92970123]


In [16]:
print("Radius Neighbours on spam data with R in [1, 100]")
print(evaluate_knn(spam_features, spam_labels, np.linspace(1, 100, 5), RNNClassifier))

Radius Neighbours on spam data with R in [1, 50]
[0.47446207 0.73723104 0.73158009 0.7148446  0.7039774 ]


In [18]:
print("Radius Neighbours on spam data with R in [20, 70]")
print(evaluate_knn(spam_features, spam_labels, np.linspace(20, 70, 5), RNNClassifier))

Radius Neighbours on spam data with R in [20, 70]
[0.74809824 0.72962399 0.73614432 0.72788524 0.71875679]


In [19]:
print("Radius Neighbours on spam data with R in [10, 45]")
print(evaluate_knn(spam_features, spam_labels, np.linspace(10, 45, 5), RNNClassifier))

Radius Neighbours on spam data with R in [10, 45]
[0.76396436 0.75527059 0.73288416 0.72875462 0.73614432]


In [20]:
print("Radius Neighbours on spam data with R in [2, 25]")
print(evaluate_knn(spam_features, spam_labels, np.linspace(2, 25, 5), RNNClassifier))

Radius Neighbours on spam data with R in [2, 25]
[0.52184308 0.76874592 0.75831341 0.7513584  0.73831776]


In [21]:
print("Radius Neighbours on spam data with R in [3, 20]")
print(evaluate_knn(spam_features, spam_labels, np.linspace(3, 20, 5), RNNClassifier))

Radius Neighbours on spam data with R in [3, 4]. Best values achieved in this interval
[0.63703543 0.77331015 0.75983482 0.7513584  0.74809824]


In [25]:
print("Radius Neighbours on spam data with R in [4, 12]. Best values achieved in this interval")
print(evaluate_knn(spam_features, spam_labels, np.linspace(4, 12, 5), RNNClassifier))

Radius Neighbours on spam data with R in [4, 12]. Best values achieved in this interval
[0.72592915 0.77613562 0.77244077 0.76396436 0.76200826]


In [22]:
normed_cancer_features = MinMaxScaler().fit_transform(cancer_features)
normed_spam_features = MinMaxScaler().fit_transform(spam_features)

In [23]:
print("kNN on normed cancer data with k in [1, 10]")
print(evaluate_knn(normed_cancer_features, cancer_labels, list(range(1, 11)), KNNClassifier))
print("kNN on normed spam data with k in [1, 10]")
print(evaluate_knn(normed_spam_features, spam_labels, list(range(1, 11)), KNNClassifier))

kNN on normed cancer data with k in [1, 10]
[0.95254833 0.95254833 0.97012302 0.97188049 0.96660808 0.96836555
 0.97012302 0.97012302 0.97012302 0.97363796]
kNN on normed spam data with k in [1, 10]
[0.912193   0.912193   0.90523799 0.91088894 0.9048033  0.91088894
 0.90067377 0.90936753 0.89806564 0.90306455]


In [24]:
print("Radius Neighbours on normed cancer data with R in [0.1, 2]")
print(evaluate_knn(normed_cancer_features, cancer_labels, np.linspace(0.1, 2, 5), RNNClassifier))

Radius Neighbours on normed cancer data with R in [0.1, 2]
[0.37258348 0.93321617 0.85764499 0.69947276 0.65026362]


In [26]:
print("Radius Neighbours on normed cancer data with R in [0.2, 1]")
print(evaluate_knn(normed_cancer_features, cancer_labels, np.linspace(0.2, 1, 5), RNNClassifier))

Radius Neighbours on normed cancer data with R in [0.2, 1]
[0.4973638  0.91212654 0.9314587  0.92091388 0.87521968]


In [30]:
print("Radius Neighbours on normed cancer data with R in [0.4, 0.8]. Best values achieved in this interval")
print(evaluate_knn(normed_cancer_features, cancer_labels, np.linspace(0.4, 0.8, 5), RNNClassifier))

Radius Neighbours on normed cancer data with R in [0.4, 0.8]. Best values achieved in this interval
[0.91212654 0.92970123 0.9314587  0.93321617 0.92091388]


In [31]:
print("Radius Neighbours on normed spam data with R in [0.05, 2]")
print(evaluate_knn(normed_spam_features, spam_labels, np.linspace(0.05, 2, 5), RNNClassifier))

Radius Neighbours on normed spam data with R in [0.05, 2]
[0.56552923 0.66746359 0.60378179 0.60617257 0.60595523]


In [32]:
print("Radius Neighbours on normed spam data with R in [0.1, 0.8]")
print(evaluate_knn(normed_spam_features, spam_labels, np.linspace(0.1, 0.8, 5), RNNClassifier))

Radius Neighbours on normed spam data with R in [0.1, 0.8]
[0.6709411  0.80221691 0.71006303 0.63116714 0.61204086]


In [34]:
print("Radius Neighbours on normed spam data with R in [0.2, 0.4]")
print(evaluate_knn(normed_spam_features, spam_labels, np.linspace(0.2, 0.4, 5), RNNClassifier))

Radius Neighbours on normed spam data with R in [0.2, 0.4]. Best values achieved in this interval
[0.79960878 0.80743317 0.79787003 0.78178657 0.74288198]


In [36]:
print("Radius Neighbours on normed spam data with R in [0.15, 0.35]. Best values achieved in this interval")
print(evaluate_knn(normed_spam_features, spam_labels, np.linspace(0.15, 0.35, 5), RNNClassifier))

Radius Neighbours on normed spam data with R in [0.15, 0.35]. Best values achieved in this interval
[0.7513584  0.79960878 0.80743317 0.79787003 0.78178657]
